In [ ]:
# function definitions are in the script, no definitions here!
include("PES_models_bonding_features.jl") # need to reload everytime a change occurs

In [ ]:
# test using actual data:
H_data = readdlm("data/h3/h3_data.txt")
# load atomic coordinates:
H_coord = npzread("data/h3/h3_coord.npy")
R = H_data[:,1:end-1]; V = H_data[:, end]
n_data = size(R)[1]

In [ ]:
idxes = shuffleobs(1:n_data); # shuffle indexes

In [ ]:
# split:
id_train, id_test = splitobs(idxes, at=0.8)
R_train = R[id_train,:]; V_train = V[id_train];
R_test = R[id_test,:]; V_test = V[id_test]
H_coord_train = H_coord[id_train,:,:]; H_coord_test = H_coord[id_test,:,:];

In [ ]:
const_r_xy = 1.4172946 # H2
#const_r_xy = 1.9369 #OH+
N = 7 # hyperparam
ρ = f_ρ(R, const_r_xy)
q = f_q_bump(N, ρ)
i = f_i(q) # should be transformed to unique integers
ϵ = f_ϵ(i, q)
# for h:
α = f_α(ϵ)
β = f_β(ϵ)
#ub = 1.; lb = -1; θ = rand(2*N+2) .* (ub-lb) .+ lb; # +2 since it starts from 0th index
θ = ones(2*N+2);

In [ ]:
#@benchmark f_\rho

In [ ]:
n_data, n_d = size(R)
u = zeros(n_data)
w = similar(u)
h = zeros(n_data, n_d, N+1) #stores the first sub primitive
compute_u_w_h!(u, w, h, θ, q, α, β, i, N)
@benchmark compute_u_w_h!($u, $w, $h, $θ, $q, $α, $β, $i, $N) #'$' for benchmark interpolation, since the vars arent const

In [ ]:
x = similar(h)
y = similar(x)
compute_hw_gw!(x, y, w, h, q, N)
@benchmark compute_hw_gw!($x, $y, $w, $h, $q, $N)